# Transforms

对数据进行变换操作，使其适合于训练  

In [51]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

In [52]:
print(ds)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()
Target transform: Lambda()


# 构建神经网络

神经网络由对数据执行操作的层/模块组成  
torch.nn提供了构建自己的神经网络所需的所有构建块    
PyTorch中的每个模块都是nn.module的子类  
神经网络本身就是一个由其他模块(层)组成的模块。这种嵌套结构允许轻松地构建和管理复杂的体系结构


在下面的部分中，我们将构建一个神经网络来对FashionMNIST数据集中的图像进行分类。

In [53]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


定义神经网络

In [54]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential
        (
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [55]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


In [56]:
# X = torch.rand(1, 28, 28, device=device)
# logits = model(X)
# pred_probab = nn.Softmax(dim=1)(logits)
# y_pred = pred_probab.argmax(1)
# print(f"Predicted class: {y_pred}")

In [57]:
input_image = torch.rand(3,28,28)
print(input_image.size())

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 28, 28])
torch.Size([3, 784])


In [58]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [59]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0184,  0.0250, -0.1660, -0.0201, -0.1665,  0.4557, -0.5192, -0.3482,
          0.0100,  0.1436, -0.3572, -0.0532, -0.3551, -0.3701, -0.4273,  0.3468,
         -0.0931, -0.3865,  0.0184,  0.0613],
        [ 0.1908, -0.1125, -0.4635,  0.1011, -0.0896,  0.2253, -0.4141, -0.0803,
          0.1238,  0.1838, -0.5379,  0.0200, -0.4270, -0.3366, -0.2064,  0.1744,
         -0.3971, -0.2857, -0.1404,  0.2262],
        [ 0.2015, -0.0749, -0.0310, -0.0132,  0.1985,  0.2930, -0.4101, -0.3140,
          0.3742,  0.2221, -0.5307,  0.1505, -0.2549, -0.3961, -0.5208,  0.0993,
         -0.2693, -0.3185, -0.1900,  0.1677]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0184, 0.0250, 0.0000, 0.0000, 0.0000, 0.4557, 0.0000, 0.0000, 0.0100,
         0.1436, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3468, 0.0000, 0.0000,
         0.0184, 0.0613],
        [0.1908, 0.0000, 0.0000, 0.1011, 0.0000, 0.2253, 0.0000, 0.0000, 0.1238,
         0.1838, 0.0000, 0.0200, 0.0000, 0.0000, 0.00